## Test inference piple

Test running images through Megadetector API to obtain object bounding boxes, then on to the MIRA API for species classification.

Adapted from http://dolphinvm.westus2.cloudapp.azure.com/ai4e/notebooks/cameratrap-sync-api-demo.html#Camera-trap-detection-API-demo

### Setup
#### 1) Make sure virtual env is running and requirements.txt are installed (see README.md)
#### 2) Load Megadetector API key from .env file
You'll need to have a .env file somewhere in the project directory with the API key set to "MEGADETECTOR_API_KEY"

In [ ]:
%load_ext dotenv
%dotenv

import os
MEGADETECTOR_API_KEY = os.getenv('MEGADETECTOR_API_KEY')

In [ ]:
import requests
import json
import matplotlib.pyplot as plt
from requests_toolbelt.multipart.encoder import MultipartEncoder

from io import BytesIO
from PIL import Image,ImageDraw
from azure.storage.blob import ContainerClient,BlobClient

%autosave 0

# Microsoft demo images:
demo_image_account_name = 'cameratrapblobs'
demo_image_container_name = 'cameratrapblobcontainer'
demo_image_account_url = 'https://' + demo_image_account_name + '.blob.core.windows.net/'
demo_image_blob_root = demo_image_account_url + demo_image_container_name
demo_image_container_client = ContainerClient(
    account_url=demo_image_account_url, 
    container_name=demo_image_container_name,
    credential=None)

# TNC test images (hosted):
test_images_remote = []
test_images_root = 'https://animl-test-images.s3-us-west-1.amazonaws.com/'
remote_image_files = [
    'sample-img-skunk-large.jpg',
    'p_001205.jpg',
    'p_001218.jpg',
]
for fil in remote_image_files:
    test_images_remote.append(test_images_root + fil)

# TNC test images (local):
test_images_local = []
test_images_dir = os.path.abspath(os.path.join(os.path.abspath(''), '..', 'input'))
local_image_files = [
    'sample-img.jpg',
    'sample-img-skunk-large.jpg',
    'sample-img-rodent.jpg',
    'sample-img-fox.jpg',
    'sample-img-fox-2.jpg',
]
for fil in local_image_files:
    test_images_local.append(os.path.join(test_images_dir, fil))

# Megadetector API config
md_api_subscription_key = MEGADETECTOR_API_KEY
md_api_base_url = 'https://aiforearth.azure-api.net/api/v1/camera-trap/sync/'
md_api_detection_url = md_api_base_url + '/detect'
md_api_version_url = md_api_base_url + '/detector_model_version'
min_confidence_to_retrieve = 0.5
min_confidence_to_display = 0.8

# MIRA API config
mira_api_url = 'https://2xuiw1fidh.execute-api.us-west-1.amazonaws.com/dev/classify'


### Megadetector API health/version check¶

In [ ]:
headers = { 'Ocp-Apim-Subscription-Key': md_api_subscription_key }
version_info = requests.get(md_api_version_url,headers=headers)
version_info.text

### Functions

In [ ]:
import urllib

def get_msft_demo_images():
    
    images_data = []
    generator = demo_image_container_client.list_blobs()

    for blob in generator:                  
        blob_client = BlobClient(demo_image_account_url,demo_image_container_name,blob.name)
        download_stream = blob_client.download_blob()
        images_data.append({'name' : blob.name, 'data': download_stream.readall()})
        # print('Read {} bytes'.format(len(images_data[-1]['data'])))
    
    return images_data   


def get_tnc_images_remote():

    images_data = []
    for url in test_images_remote:
        with urllib.request.urlopen(url) as img:
            images_data.append({'name': url, 'data': BytesIO(img.read()).read()})
            
    return images_data


def get_tnc_images_local():

    images_data = []
    for fil in test_images_local:
        with open(fil, "rb") as img:
            images_data.append({'name': fil, 'data': img.read()})
            
    return images_data


def draw_bounding_box_on_image(image,ymin,xmin,ymax,xmax):

    color = 'red'
    draw = ImageDraw.Draw(image)
    im_width, im_height = image.size
    (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                  ymin * im_height, ymax * im_height)
    draw.line([(left, top), (left, bottom), (right, bottom),
               (right, top), (left, top)], width=4, fill=color)
    

    
def draw_raw_images():  
    
    fig = plt.figure(figsize=(20,35))

    num_images = len(images)
    
    columns = 2

    rows = (num_images // 2) + (num_images % 2)
    
    for i in range(len(images)):
        
        image = Image.open(BytesIO(images[i]['data']))        
        axis = plt.subplot(rows,columns, i + 1)     
        axis.imshow(image)
        plt.axis('off')
        plt.axis('tight')
            
    plt.show()


def response_to_json(r):
    
    from requests_toolbelt.multipart import decoder
    res = decoder.MultipartDecoder.from_response(r)
    results = {}
    images = {}

    for part in res.parts:
        
        # 'part' is a BodyPart object with b'Content-Type', and b'Content-Disposition';
        # the latter includes 'name' and 'filename' info

        headers = {}
        for k, v in part.headers.items():
            headers[k.decode(part.encoding)] = v.decode(part.encoding)

        if headers.get('Content-Type', None) == 'image/jpeg':
            c = headers.get('Content-Disposition')
            image_name = c.split('name="')[1].split('"')[0]
            image = Image.open(io.BytesIO(part.content))
            images[image_name] = image
        elif headers.get('Content-Type', None) == 'application/json':
            content_disposition = headers.get('Content-Disposition', '')
            if 'detection_result' in content_disposition:
                results['detection_result'] = json.loads(part.content.decode())
            elif 'classification_result' in content_disposition:
                results['classification_result'] = json.loads(part.content.decode())
                
    # ...for each part
    
    return results,images


def call_megadetector(image_info):
    
    image_bytes = image_info['data']
    image_name = image_info['name']
    
    assert isinstance(image_bytes,bytes)
    file = BytesIO(image_bytes)
    files = {}
    files[image_name] = (image_name, file, 'application/octet-stream')
    
    headers = { 'Ocp-Apim-Subscription-Key': md_api_subscription_key }
    params = {
        'confidence': min_confidence_to_retrieve,
        'render': False
    }
    
    r = requests.post(md_api_detection_url, params=params, headers=headers, files=files)
    
    return r


def call_mira(image_info):

    image_bytes = image_info.get('data')
    image_name = image_info.get('name')
    image_bbox = image_info.get('bbox')

    fields = {}
    fields['image'] = (image_name, image_bytes, 'image/jpeg')
    fields['bbox'] = json.dumps(image_bbox)
    
    multipart_data = MultipartEncoder(fields = fields)
    r = requests.post(mira_api_url,
                      data = multipart_data,
                      headers = {'Content-Type': multipart_data.content_type})
    
    return r

### Retrieve sample images
Uncomment the image fetching function you want & comment out the rest.

In [ ]:
# images = get_msft_demo_images()
# images = get_tnc_images_local()
images = get_tnc_images_remote()

draw_raw_images()

### Select an image

In [ ]:
i_image = 2

### Run inference

In [ ]:
# Call Megadetector 

# Each detection is: [ymin, xmin, ymax, xmax, confidence, class]
# Image coordinates are normalized, with the origin in the upper-left

r = call_megadetector(images[i_image])

# print('megadetector response code: {}'.format(r))
results,_ = response_to_json(r)
image_name = images[i_image]['name']
detections = results['detection_result'][image_name]
print('Megadetector found {} object(s) in {}'.format(len(detections), image_name))
print('{}\n'.format(detections))

# Add bounding box to image dict and call MIRA
images[i_image]['bbox'] = detections[0][:4]
r = call_mira(images[i_image])

# print('mira response code: {}'.format(r))
# print('response: {}'.format(r.json()))
for key, value in r.json().items():
    print('{} predictions: '.format(key))
    for classification, pred in value.get('predictions').items():
        print('{}: {}'.format(classification, pred))
    print('\n')

### Show results

In [ ]:
from PIL import Image
image = Image.open(BytesIO(images[i_image]['data']))

for detection in detections:
    box = detection[0:4]
    confidence = detection[4]
    clss = detection[5]
    if (confidence >= min_confidence_to_display):
        draw_bounding_box_on_image(image, box[0], box[1], box[2], box[3])
image